<a href="https://colab.research.google.com/github/thinhngo-x/Anonymization/blob/master/notebook/Anonymization_XLNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet informatique 9

## GDPR in practice: data anonymization

This quick notebook will show you how to anonymize text data automatically. You'll have to do the same, by testing other approaches and / or embeddings and / or classification algorithms.

### Some dependencies and some data

In [0]:
from google.colab import drive
drive.mount('/content/drive')
root = '/content/drive/My Drive/Anonymization/CONLL2003_XLNET/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
! pip install transformers

     |████████████████████████████████| 645kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 13.5MB/s 
     |████████████████████████████████| 3.8MB 17.4MB/s 
     |████████████████████████████████| 890kB 30.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=05325e73e3c210cf21507b02ea6098aba421b1f611ee6d677087891db9939de4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
import transformers
import os
from torch.nn import CrossEntropyLoss
import logging
logger = logging.getLogger(__name__)

In [0]:
from urllib.request import urlopen
! wget -nc "https://raw.githubusercontent.com/glample/tagger/master/dataset/eng.testa"
! wget -nc "https://raw.githubusercontent.com/glample/tagger/master/dataset/eng.train"
! wget -nc "https://raw.githubusercontent.com/glample/tagger/master/dataset/eng.testb"
# html = urlopen(url)
# with open('eng.testb', 'b') as the_file:
#     for line in html:
#         the_file.write(line.decode('UTF-8'))

--2020-05-17 17:59:08--  https://raw.githubusercontent.com/glample/tagger/master/dataset/eng.testa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 827009 (808K) [text/plain]
Saving to: ‘eng.testa’

eng.testa           100%[===================>] 807.63K  --.-KB/s    in 0.08s   

2020-05-17 17:59:09 (9.33 MB/s) - ‘eng.testa’ saved [827009/827009]

--2020-05-17 17:59:10--  https://raw.githubusercontent.com/glample/tagger/master/dataset/eng.train
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3281527 (3.1M) [text/plain]
Saving to: ‘eng.train’


### Some useful functions

In [0]:
class InputExample(object):
    """A single training/test example for token classification."""

    def __init__(self, guid, words, labels):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            words: list. The words of the sequence.
            labels: (Optional) list. The labels for each word of the sequence. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.words = words
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids


def read_examples_from_file(data_dir, mode="eng.testa"):
    file_path = os.path.join(data_dir, "{}".format(mode))
    guid_index = 1
    examples = []
    with open(file_path, encoding="utf-8") as f:
        words = []
        labels = []
        for line in f:
            if line.startswith("-DOCSTART-") or line == "" or line == "\n":
                if words:
                    examples.append(InputExample(guid="{}-{}".format(mode, guid_index), words=words, labels=labels))
                    guid_index += 1
                    words = []
                    labels = []
            else:
                splits = line.split(" ")
                words.append(splits[0])
                if len(splits) > 1:
                    labels.append(splits[-1].replace("\n", ""))
                else:
                    # Examples could have no label for mode = "test"
                    labels.append("O")
        if words:
            examples.append(InputExample(guid="{}-{}".format(mode, guid_index), words=words, labels=labels))
    return examples


def convert_examples_to_features(
    examples,
    label_list,
    max_seq_length,
    tokenizer,
    cls_token_at_end=False,
    cls_token="[CLS]",
    cls_token_segment_id=1,
    sep_token="[SEP]",
    sep_token_extra=False,
    pad_on_left=False,
    pad_token=0,
    pad_token_segment_id=0,
    pad_token_label_id=-100,
    sequence_a_segment_id=0,
    mask_padding_with_zero=True,
):
    """ Loads a data file into a list of `InputBatch`s
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """

    label_map = {label: i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            logger.info("Writing example %d of %d", ex_index, len(examples))

        tokens = []
        label_ids = []
        for word, label in zip(example.words, example.labels):
            word_tokens = tokenizer.tokenize(word)
            tokens.extend(word_tokens)
            # Use the real label id for the first token of the word, and padding ids for the remaining tokens
            label_ids.extend([label_map[label]] + [pad_token_label_id] * (len(word_tokens) - 1))

        # Account for [CLS] and [SEP] with "- 2" and with "- 3" for RoBERTa.
        special_tokens_count = 3 if sep_token_extra else 2
        if len(tokens) > max_seq_length - special_tokens_count:
            tokens = tokens[: (max_seq_length - special_tokens_count)]
            label_ids = label_ids[: (max_seq_length - special_tokens_count)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids:   0   0   0   0  0     0   0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens += [sep_token]
        label_ids += [pad_token_label_id]
        if sep_token_extra:
            # roberta uses an extra separator b/w pairs of sentences
            tokens += [sep_token]
            label_ids += [pad_token_label_id]
        segment_ids = [sequence_a_segment_id] * len(tokens)

        if cls_token_at_end:
            tokens += [cls_token]
            label_ids += [pad_token_label_id]
            segment_ids += [cls_token_segment_id]
        else:
            tokens = [cls_token] + tokens
            label_ids = [pad_token_label_id] + label_ids
            segment_ids = [cls_token_segment_id] + segment_ids

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_seq_length - len(input_ids)
        if pad_on_left:
            input_ids = ([pad_token] * padding_length) + input_ids
            input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
            segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
            label_ids = ([pad_token_label_id] * padding_length) + label_ids
        else:
            input_ids += [pad_token] * padding_length
            input_mask += [0 if mask_padding_with_zero else 1] * padding_length
            segment_ids += [pad_token_segment_id] * padding_length
            label_ids += [pad_token_label_id] * padding_length

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        assert len(label_ids) == max_seq_length

        if ex_index < 5:
            logger.info("*** Example ***")
            logger.info("guid: %s", example.guid)
            logger.info("tokens: %s", " ".join([str(x) for x in tokens]))
            logger.info("input_ids: %s", " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s", " ".join([str(x) for x in input_mask]))
            logger.info("segment_ids: %s", " ".join([str(x) for x in segment_ids]))
            logger.info("label_ids: %s", " ".join([str(x) for x in label_ids]))

        features.append(
            InputFeatures(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids, label_ids=label_ids)
        )
    return features

### A direct, pre-trained NER approach

In [0]:
nlp = transformers.pipeline("ner")

In [0]:
nlp("Adrien Ehrhardt donne un projet d'informatique aux étudiants de INF442.")

[{'entity': 'I-PER', 'score': 0.9998049736022949, 'word': 'Ad'},
 {'entity': 'I-PER', 'score': 0.9996461272239685, 'word': '##rien'},
 {'entity': 'I-PER', 'score': 0.9998077154159546, 'word': 'E'},
 {'entity': 'I-PER', 'score': 0.992250919342041, 'word': '##hr'},
 {'entity': 'I-PER', 'score': 0.9994708299636841, 'word': '##hardt'},
 {'entity': 'I-ORG', 'score': 0.8828932642936707, 'word': 'IN'},
 {'entity': 'I-ORG', 'score': 0.5023669600486755, 'word': '##F'}]

### Obtaining the representation of the dataset from Bert

#### An example

In [0]:
from transformers import XLNetTokenizer, XLNetModel
import torch

In [0]:
model = XLNetModel.from_pretrained("xlnet-large-cased").cuda()
tokenizer = XLNetTokenizer.from_pretrained("xlnet-large-cased")

sequence = "Adrien Ehrhardt donne un projet d'informatique aux étudiants de INF442."
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
inputs = tokenizer.encode(sequence, return_tensors="pt")

outputs = model(inputs.cuda())[0]

In [0]:
outputs.detach().cpu().numpy()[0].shape

(33, 1024)

In [0]:
outputs

tensor([[[-0.8624, -0.1178,  0.8844,  ..., -2.1735,  0.8973,  0.7981],
         [ 1.7510, -1.6261, -1.2900,  ..., -1.2988, -0.4977,  0.0453],
         [ 0.2788, -1.3114,  0.1634,  ...,  0.4603,  0.4635,  2.2995],
         ...,
         [ 0.1434, -0.6837,  0.9015,  ..., -1.7662,  0.5002,  0.3987],
         [-0.0049, -0.3282,  0.9206,  ..., -1.1432, -0.5351, -1.1123],
         [ 0.2574, -0.2640,  1.1133,  ..., -0.9950, -0.2340, -1.0951]]],
       device='cuda:0', grad_fn=<PermuteBackward>)

In [0]:
del outputs

#### The real thing

In [0]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset

In [0]:
from tqdm.notebook import tqdm, trange

In [0]:
import numpy as np

In [0]:
pad_token_label_id=CrossEntropyLoss().ignore_index
labels = ["O", "B-MISC", "I-MISC", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]
model = XLNetModel.from_pretrained("xlnet-large-cased").cuda()
tokenizer = XLNetTokenizer.from_pretrained("xlnet-large-cased")
examples = read_examples_from_file(".", mode="eng.train")
features = convert_examples_to_features(
            examples,
            label_list = labels,
            max_seq_length = 128,
            tokenizer = tokenizer,
            cls_token_at_end=True,
            cls_token=tokenizer.cls_token,
            cls_token_segment_id=2,
            sep_token=tokenizer.sep_token,
            sep_token_extra=False,
            pad_on_left=False,
            pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
            pad_token_segment_id=0,
            pad_token_label_id=pad_token_label_id)

In [0]:
# Convert to Tensors and build dataset
all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_ids for f in features], dtype=torch.long)

dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

In [0]:
eval_sampler = SequentialSampler(dataset)
eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=32)
preds = None
out_label_ids = None

In [0]:
model.eval()
preds = []
out_label_ids = []
for batch in tqdm(eval_dataloader, desc="Evaluating"):
    batch = tuple(t.to("cuda") for t in batch)

    with torch.no_grad():
        inputs = {"input_ids": batch[0], "attention_mask": batch[1]}
        inputs["token_type_ids"] = (
            batch[2]
        )
        outputs = model(**inputs)
        last_hidden_layer = outputs[0]
        # if preds is None:
        #     preds = last_hidden_layer.detach().cpu().numpy()
        #     out_label_ids = batch[3].detach().cpu().numpy()
        # else:
        #     preds = np.append(preds, last_hidden_layer.detach().cpu().numpy(), axis=0)
        #     out_label_ids = np.append(out_label_ids, batch[3].detach().cpu().numpy(), axis=0)
        #     print(last_hidden_layer.shape)
        #     print(batch[3].shape)
        preds.append(last_hidden_layer.detach().cpu().numpy())
        out_label_ids.append(batch[3].detach().cpu().numpy())
        # preds[ite*32:(ite+1)*32,:,:] = last_hidden_layer.detach().cpu().numpy()
        # out_label_ids[ite*32:(ite+1)*32,:,:] = out_label_ids.detach().cpu().numpy()
        # ite += 1

In [0]:
out_label_ids[-1].shape

(25, 128)

In [0]:
out_label_list = [[] for _ in range(len(out_label_ids) * 32)]
preds_list = [[] for _ in range(len(out_label_ids) * 32)]

label_map = {i: label for i, label in enumerate(labels)}
        
for i in range(len(out_label_ids)):
    for b in range(out_label_ids[i].shape[0]):
        for j in range(out_label_ids[0].shape[1]):
            if out_label_ids[i][b,j] != pad_token_label_id:
                out_label_list[i*32+b].append(out_label_ids[i][b,j])
                preds_list[i*32+b].append(preds[i][b,j])

In [0]:
out_label_list

[[6, 0, 2, 0, 0, 0, 2, 0, 0],
 [4, 4],
 [8, 0],
 [0,
  6,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  0,
  0,
  0,
  0,
  0,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [8,
  0,
  0,
  0,
  0,
  6,
  6,
  0,
  0,
  0,
  4,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  6,
  0,
  0,
  0,
  4,
  4,
  4,
  4,
  0,
  0,
  0,
  0,
  0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  6,
  0,
  0,
  4,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [4,
  0,
  2,
  0,
  0,
  0,
  0,
  8,
  0,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  2,
  2,
  0,
  2,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  6,
  0,
  0,

`out_label_list` is of shape (#number of sentences, #tokens in this sentence, 1)

`preds_list` is of shape (#number of sentences, #tokens in this sentence, 1024 hidden representations)

For the sake of simplicity, we'll consider that an observation / a sample is a token, not a sentence (the contextual meaning of each token is already taken into account in its representation), so we need to "flatten" both lists so that they're of shape (#tokens, 1)

In [0]:
import itertools

In [0]:
true_labels = np.array(list(itertools.chain.from_iterable(out_label_list)))

In [0]:
flat_list = []
for sentence in preds_list:
    for token in sentence:
        flat_list.append(token)
        
representation = np.array(flat_list)

In [0]:
true_labels.shape

(203583,)

In [0]:
representation.shape

(203583, 1024)

In [0]:
np.save(root+"true_labels.train.npy", true_labels)

In [0]:
np.save(root+"representation.train.npy", representation)

### Example of classifier on top

In [0]:
import sklearn as sk
import sklearn.linear_model

In [0]:
model_top = sk.linear_model.LogisticRegressionCV()

In [0]:
model_top.fit(X=representation, y=true_labels)

/Users/adimajo/opt/anaconda3/envs/inf442_PI9/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/adimajo/opt/anaconda3/envs/inf442_PI9/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sc

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='auto', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

In [0]:
model_top.score(X=representation, y=true_labels)

0.9777055203273197

### Loading for example

In [0]:
true_labels = np.load("true_labels.train.npy")

In [0]:
true_labels

array(['I-ORG', 'O', 'I-MISC', ..., 'O', 'I-ORG', 'O'], dtype='<U6')

In [0]:
representation = np.load("representation.train.npy")

In [0]:
representation

array([[-0.5649642 , -0.5623049 , -0.3155526 , ..., -0.14965823,
         0.2672749 , -0.00221438],
       [-0.3143751 ,  1.3328775 ,  0.91796803, ...,  0.9563622 ,
         0.30265984, -0.38435107],
       [-0.87775815,  0.2750554 ,  0.41846916, ...,  0.0536996 ,
         0.08224589,  0.489617  ],
       ...,
       [-0.15607172,  0.28895253,  0.45071855, ...,  0.349473  ,
         0.13088518,  0.4136334 ],
       [-0.14589024,  0.46420857, -0.1206243 , ..., -0.7722274 ,
        -0.07450192,  0.7521924 ],
       [-0.39739043,  0.15462524, -0.00154328, ...,  0.13841854,
         0.27207375,  0.56779885]], dtype=float32)

### Shrinking attempt

In [0]:
# from scipy import sparse

In [0]:
# representation_sparse = sparse.csr_matrix(representation)

In [0]:
# sparse.save_npz("representation.train.npz", representation_sparse)

Not successfull!

### Loading and converting to CSV: files get too big, you'll have to do this yourself and sample!

In [0]:
from random import sample

In [0]:
representation = np.load("representation.train.npy")
# SAMPLE
int_train = sample(range(representation.shape[0]), 10000)
np.savetxt("representation.train.csv", representation[int_train,:], delimiter=",")
true_labels = np.load("true_labels.train.npy")
np.savetxt("true_labels.train.csv", true_labels[int_train], delimiter=",", fmt='%s')

In [0]:
representation = np.load("representation.testa.npy")
# SAMPLE
int_train = sample(range(representation.shape[0]), 2000)
np.savetxt("representation.testa.csv", representation[int_train,:], delimiter=",")
true_labels = np.load("true_labels.testa.npy")
np.savetxt("true_labels.testa.csv", true_labels[int_train], delimiter=",", fmt='%s')

In [0]:
representation = np.load("representation.testb.npy")
# SAMPLE
int_train = sample(range(representation.shape[0]), 2000)
np.savetxt("representation.testb.csv", representation[int_train,:], delimiter=",")
true_labels = np.load("true_labels.testb.npy")
np.savetxt("true_labels.testb.csv", true_labels[int_train], delimiter=",", fmt='%s')